## Rebalancing

In [1]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from scipy.optimize import minimize
from cvxopt import matrix
from cvxopt.solvers import qp as Solver, options as SolverOptions
from scipy.stats import multivariate_normal as mvn

pd.options.display.float_format = '{:,.4f}'.format

### Two asset simulation with no rebalancing

In [2]:
# Single time-series of returns
def returns(mn1, mn2, sd1, sd2, c, rf, init_wt1, init_wt2, T):
    # Organize inputs
    mns = [mn1, mn2]
    sds = [sd1, sd2]   
    cov = np.array([[sds[0] ** 2, sds[0] * sds[1] * c], [sds[0] * sds[1] * c, sds[1] ** 2]]).reshape(2, 2)

    # Generate random sample and do calculations
    col_names = ['beg_wgt1','beg_wgt2','ret1','ret2', 'end_wgt1','end_wgt2','realized_pret','exp_pret','sd_pret','sharpe ratio']
    df = pd.DataFrame(dtype=float, columns=col_names, index=np.arange(T)+1)
    df[['ret1','ret2']]  = mvn.rvs(mns, cov, size=T)
    df.loc[1,'beg_wgt1'] = init_wt1
    df.loc[1,'beg_wgt2'] = init_wt2
    for t in df.index:
        if t > 1:
            df.loc[t,'beg_wgt1'] = df.loc[t-1,'end_wgt1']
            df.loc[t,'beg_wgt2'] = df.loc[t-1,'end_wgt2']
        # Calculate realized portfolio return
        w1 = df.loc[t,'beg_wgt1']
        w2 = df.loc[t,'beg_wgt2']
        r1 = df.loc[t,'ret1']
        r2 = df.loc[t,'ret2']
        rp = w1*r1 + w2*r2
        df.loc[t,'realized_pret']=rp    
        # Update weights
        df.loc[t,'end_wgt1']  = w1 * (1+r1)/(1+rp)
        df.loc[t,'end_wgt2']  = w2 * (1+r2)/(1+rp)
    df['exp_pret'] = df.beg_wgt1*mns[0] + df.beg_wgt2*mns[1]
    df['sd_pret']  = np.sqrt(df.beg_wgt1**2 * sds[0]**2  + df.beg_wgt2**2 * sds[1]**2 + 2*df.beg_wgt1*df.beg_wgt2*c*sds[0]*sds[1])        
    df['sharpe ratio'] = (df.exp_pret - rf) / df.sd_pret
    return df

Let's look at a single realization

In [3]:
# Inputs
MN_STOCK = 6  / 100
MN_BOND  = 3.5/ 100
SD_STOCK = 15 / 100
SD_BOND  = 3.5/ 100
CORR = -5 / 100
RF   = 3.4364 / 100
INIT_WGT_STOCK = 0.6
INIT_WGT_BOND  = 0.4
T = 30

# A single realization
rets = returns(MN_STOCK, MN_BOND, SD_STOCK, SD_BOND, CORR, RF, INIT_WGT_STOCK, INIT_WGT_BOND, T)
rets

,beg_wgt1,beg_wgt2,ret1,ret2,end_wgt1,end_wgt2,realized_pret,exp_pret,sd_pret,sharpe ratio
1,0.6000,0.4000,0.0572,0.0216,0.6082,0.3918,0.0430,0.0500,0.0904,0.1730
2,0.6082,0.3918,-0.0014,0.0741,0.5907,0.4093,0.0282,0.0502,0.0916,0.1730
3,0.5907,0.4093,0.0845,0.0757,0.5927,0.4073,0.0809,0.0498,0.0890,0.1730
4,0.5927,0.4073,0.2056,-0.0136,0.6400,0.3600,0.1163,0.0498,0.0893,0.1730
5,0.6400,0.3600,-0.0868,-0.0178,0.6231,0.3769,-0.0620,0.0510,0.0962,0.1729
6,0.6231,0.3769,0.0277,-0.0110,0.6321,0.3679,0.0131,0.0506,0.0937,0.1730
7,0.6321,0.3679,0.0721,0.0094,0.6460,0.3540,0.0490,0.0508,0.0950,0.1730
8,0.6460,0.3540,0.4061,0.0170,0.7162,0.2838,0.2684,0.0512,0.0971,0.1729
9,0.7162,0.2838,0.1930,0.0861,0.7349,0.2651,0.1627,0.0529,0.1074,0.1726
10,0.7349,0.2651,0.1066,0.0327,0.7481,0.2519,0.0870,0.0534,0.1102,0.1726


Let's simulate 1000 realizations of 30 years of investing and collect the realized sharpe ratios

- We need to update our function to return the realized Sharpe ratio (Note: Sharpe ratio column above is forward-looking SR)
- We need to collect the results of the 1000 simulations

In [4]:
# Realized SR for a single realization
(rets.realized_pret.mean()-RF)/rets.realized_pret.std()

0.47477997863223304

In [5]:
# Simulate a single realization and do calculation(s)
def sim_calc(mn1, mn2, sd1, sd2, c, rf, init_wt1, init_wt2, T):
    # Organize inputs
    mns = [mn1, mn2]
    sds = [sd1, sd2]   
    cov = np.array([[sds[0] ** 2, sds[0] * sds[1] * c], [sds[0] * sds[1] * c, sds[1] ** 2]]).reshape(2, 2)

    # Generate random sample and do calculations
    col_names = ['beg_wgt1','beg_wgt2','ret1','ret2', 'end_wgt1','end_wgt2','realized_pret','exp_pret','sd_pret','sharpe ratio']
    df = pd.DataFrame(dtype=float, columns=col_names, index=np.arange(T)+1)
    df[['ret1','ret2']]  = mvn.rvs(mns, cov, size=T)
    df.loc[1,'beg_wgt1'] = init_wt1
    df.loc[1,'beg_wgt2'] = init_wt2
    for t in df.index:
        if t > 1:
            df.loc[t,'beg_wgt1'] = df.loc[t-1,'end_wgt1']
            df.loc[t,'beg_wgt2'] = df.loc[t-1,'end_wgt2']
        # Calculate realized portfolio return
        w1 = df.loc[t,'beg_wgt1']
        w2 = df.loc[t,'beg_wgt2']
        r1 = df.loc[t,'ret1']
        r2 = df.loc[t,'ret2']
        rp = w1*r1 + w2*r2
        df.loc[t,'realized_pret']=rp    
        # Update weights
        df.loc[t,'end_wgt1']  = w1 * (1+r1)/(1+rp)
        df.loc[t,'end_wgt2']  = w2 * (1+r2)/(1+rp)
    df['exp_pret'] = df.beg_wgt1*mns[0] + df.beg_wgt2*mns[1]
    df['sd_pret']  = np.sqrt(df.beg_wgt1**2 * sds[0]**2  + df.beg_wgt2**2 * sds[1]**2 + 2*df.beg_wgt1*df.beg_wgt2*c*sds[0]*sds[1])        
    df['sharpe ratio'] = (df.exp_pret - rf) / df.sd_pret
    return (df.realized_pret.mean() - rf)/df.realized_pret.std()


In [6]:
# Collect N_SIMS runs of the simulation function
N_SIMS=1000
sims = pd.DataFrame(dtype=float,columns=['sharpe'],index=np.arange(N_SIMS))
for s in sims.index:
    sims.loc[s] = sim_calc(MN_STOCK, MN_BOND, SD_STOCK, SD_BOND, CORR, RF, INIT_WGT_STOCK, INIT_WGT_BOND, T)

In [7]:
# Plot the distribution of output
fig = go.Figure()
trace= go.Histogram(x=sims.sharpe, histnorm='percent',hovertemplate="<br>%{y:.2}% of simulations <br><extra></extra>")
fig.add_trace(trace)
fig.update_traces(marker_line_width=1, marker_line_color='black')
fig.layout.xaxis["title"] = "Realized Sharpe Ratio"
fig.layout.yaxis["title"] = "Percent of Simulations"
fig.show()

### N asset simulation with no rebalancing

In [8]:
def returns(means, sds, corr, rf, init_wgts, T):
    # Inputs    
    cov = np.diag(sds) @ corr @ np.diag(sds)
    n = len(means)
    
    # Set up dataframe
    beg_wgt_names = ['beg_wgt'+str(i) for i in np.arange(n)]
    end_wgt_names = ['end_wgt'+str(i) for i in np.arange(n)]
    ret_names = ['ret'+str(i) for i in np.arange(n)]
    col_names = beg_wgt_names + ret_names + end_wgt_names +['realized_pret','exp_pret','sd_pret','sharpe ratio']
    df = pd.DataFrame(dtype=float, columns=col_names, index=np.arange(T)+1)

    # Generate random sample and do calculations
    df[ret_names] = mvn.rvs(means, cov, size=T)
    df.loc[1,beg_wgt_names] = init_wgts
    for t in df.index:
        if t > 1:
            df.loc[t,beg_wgt_names] = df.loc[t-1,end_wgt_names].values
        # Calculate realized portfolio return + beginning of period E[r] and SD[r]
        wgts = df.loc[t,beg_wgt_names].values
        rets = df.loc[t,ret_names].values
        rp = wgts @ rets
        df.loc[t,'realized_pret']=rp        
        df.loc[t,'exp_pret'] = wgts @ means
        df.loc[t,'sd_pret']  = np.sqrt(wgts @ cov @ wgts)         
        # Update weights
        for i in np.arange(n):
            w = df.loc[t,'beg_wgt'+str(i)]
            r = df.loc[t,'ret'+str(i)]
            df.loc[t,'end_wgt'+str(i)]  =  w * (1+r)/(1+rp)
    df['sharpe ratio'] = (df.exp_pret - rf) / df.sd_pret
    return df

Inputs

In [9]:
# Risk-free rate
RF = 0.01

# Expected returns
MNS = np.array([0.06, 0.065, 0.08])

# Standard deviations
SDS = np.array([0.15, 0.165, 0.21])

# Correlations
C  = np.identity(3)
C[0, 1] = C[1, 0] = 0.75
C[0, 2] = C[2, 0] = 0.75
C[1, 2] = C[2, 1] = 0.75

# Sample length
T = 30

A single realization

In [10]:
n = len(MNS)
df = returns(MNS, SDS, C, RF, (1/n)*np.ones(n), T)
df.head()

,beg_wgt0,beg_wgt1,beg_wgt2,ret0,ret1,ret2,end_wgt0,end_wgt1,end_wgt2,realized_pret,exp_pret,sd_pret,sharpe ratio
1,0.3333,0.3333,0.3333,0.0437,0.0106,0.1172,0.3291,0.3186,0.3523,0.0572,0.0683,0.1599,0.3648
2,0.3291,0.3186,0.3523,-0.1156,0.0076,-0.0136,0.3033,0.3346,0.3621,-0.0404,0.0686,0.1609,0.3645
3,0.3033,0.3346,0.3621,-0.0051,-0.0776,-0.0172,0.3123,0.3194,0.3683,-0.0338,0.0689,0.1617,0.3643
4,0.3123,0.3194,0.3683,0.3596,0.1598,0.1806,0.3452,0.3012,0.3536,0.2299,0.0690,0.1619,0.3642
5,0.3452,0.3012,0.3536,-0.1279,0.0049,-0.2347,0.3444,0.3462,0.3095,-0.1256,0.0686,0.1607,0.3645


### N asset simulation with rebalancing to constant weights

- With rebalancing to constant weights, we do not need to track weights each period

In [11]:
def returns_rebalancing(means, sds, corr, rf, wgts, T):
    cov = np.diag(sds) @ corr @ np.diag(sds)
    n = len(means)
    
    # Set up dataframe
    ret_names = ['ret'+str(i) for i in np.arange(n)]
    col_names = ret_names + ['realized_pret','exp_pret','sd_pret','sharpe ratio']
    df = pd.DataFrame(dtype=float, columns=col_names, index=np.arange(T)+1)

    # Generate random sample and do calculations
    df[ret_names] = mvn.rvs(means, cov, size=T)
    for t in df.index:
        df.loc[t,'exp_pret'] = wgts @ means
        df.loc[t,'sd_pret']  = np.sqrt(wgts @ cov @ wgts)  
        rets = df.loc[t,ret_names].values
        df.loc[t,'realized_pret']=wgts @ rets 
    df['sharpe ratio'] = (df.exp_pret - rf) / df.sd_pret
    return df

A single realization

In [12]:
n = len(MNS)
df = returns_rebalancing(MNS, SDS, C, RF, (1/n)*np.ones(n), T)
df.head()

,ret0,ret1,ret2,realized_pret,exp_pret,sd_pret,sharpe ratio
1,0.0144,0.1803,0.1215,0.1054,0.0683,0.1599,0.3648
2,-0.1968,0.1288,-0.1079,-0.0586,0.0683,0.1599,0.3648
3,-0.1262,-0.1384,-0.1558,-0.1402,0.0683,0.1599,0.3648
4,0.3170,0.3527,0.2288,0.2995,0.0683,0.1599,0.3648
5,0.2730,0.1953,0.1805,0.2163,0.0683,0.1599,0.3648
